In [17]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

In [18]:
import os
# Office
os.chdir('C:\\Users\\prudi\\Documents\\GitHub\\Code\\NITDATA\\Algorithms\\codesnippetsforbaggingandboostingalgorithms')
data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

class PreProcessing(BaseEstimator, TransformerMixin):


    def __init__(self):
        pass

    
    ## Here this model will be called with a variable (df), which is a dataframe
    def datapreprocessing(self, df):
        
        df = df.reset_index(drop=True)
        
        df.loc[df['Gender'].isnull(),'Gender']='Trasgender'
        df.loc[df['Dependents'].isnull(),'Dependents']='0'
        df.loc[df['Education'].isnull(),'Education']='No Education'
        df.loc[df['Married'].isnull(),'Married']='No'
        df.loc[df['Self_Employed'].isnull(),'Self_Employed']='No'
        df.loc[df['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
        df.loc[df['Credit_History'].isnull(),'Credit_History']=0
        df.loc[df['LoanAmount'].isnull(),'LoanAmount']=146.867     
        df['Credit_History']=df['Credit_History'].astype(str)
        
        
        ## Creating a Instance for the minmax Pickle File
        MinMaxpickle_in = open("minmax_pickle.pkl","rb")
        MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        
        ## Applying the pickle file
        df['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].transform(df[['ApplicantIncome']]))
        
        ## Creating a Instance for the minmax Pickle File
        standardscaler_in = open("standardscaler_pickle.pkl","rb")
        standardscaler_dict = pickle.load(standardscaler_in)
        
        ## Applying the pickle file
        df['LoanAmount']=pd.DataFrame(standardscaler_dict['LoanAmount'].transform(df[['LoanAmount']]))
        
        ## Creating a Instance for the LabelEncoder Pickle File
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Education']=pd.DataFrame(Labelencoder_dict['Education'].transform(df[['Education']]))
        df['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].transform(df[['Property_Area']]))
        df['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].transform(df[['Credit_History']]))
        df['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].transform(df[['Dependents']]))
        
        ## Creating a Instance for the binarizer Pickle File
        Onehotpickle_in = open("binarizer_pickle.pkl","rb")
        Onehot_dict = pickle.load(Onehotpickle_in)
        
        ## Applying the pickle file
        d1=pd.DataFrame(Onehot_dict['Self_Employed'].transform(df[['Self_Employed']]))
        d1.columns=['Self_Employed_0']
        d2=pd.DataFrame(Onehot_dict['Married'].transform(df[['Married']]))
        d2.columns=['Married_0']
        d3=pd.DataFrame(Onehot_dict['Gender'].transform(df[['Gender']]))
        d3.columns=['Gender_0','Gender_1','Gender_2']
    
        df=df.drop(['Self_Employed','Married','Gender','Loan_ID'],axis=1)
        df=pd.concat([df,d1,d2,d3],axis=1)
        
        return df.as_matrix()

    def definingvalues(self, df, y=None,**fit_params):
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self
    
    def encodingTargetVariable(self,df):
        Labelencoderpickle_in = open("labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        
        ## Applying the pickle file
        df['Loan_Status']=pd.DataFrame(Labelencoder_dict['Loan_Status'].transform(df[['Loan_Status']]))
        
        return df.as_matrix()

In [20]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [21]:
pred_var = ['Loan_ID','Gender','Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']
X_train, X_test, y_train, y_test = train_test_split(data[pred_var], data['Loan_Status'], test_size=0.30, random_state=42)

In [22]:
## Creating an instance for the class
preprocess = PreProcessing()

In [23]:
# Calling the fit function present in the class Preprocessing
#preprocess.definingvalues(X_train)

PreProcessing()

In [24]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_train dataset
data_transformed_xtrain = preprocess.datapreprocessing(X_train)

pred_var = ['Dependents', 'Education', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area', 'Self_Employed_0', 'Married_0', 'Gender_0', 'Gender_1','Gender_2']


data_transformed_xtrain=pd.DataFrame(data_transformed_xtrain,columns=pred_var)
data_transformed_xtrain.head()

print(data_transformed_xtrain.shape)
print(X_train.shape)

(429, 13)
(429, 12)


In [32]:
data_transformed_xtrain

,Dependents,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Self_Employed_0,Married_0,Gender_0,Gender_1,Gender_2
0,0.0,1.0,0.175717,2333.0,0.689937,360.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.209274,2792.0,0.282643,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.145668,2209.0,-0.046325,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.186852,1700.0,-0.469284,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.505872,0.0,-0.108986,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2.0,1.0,0.131559,1875.0,-0.625936,360.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
425,2.0,0.0,0.859289,1126.0,1.379203,360.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0
426,0.0,0.0,0.235433,0.0,-1.675501,360.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0
427,0.0,0.0,0.754805,0.0,0.155238,240.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [26]:
## Calling the datapreprocessing function present in the class Preprocessing to Preprocess X_test dataset
data_transformed_xtest = preprocess.datapreprocessing(X_test)

# Converting the matrix to a dataframe
data_transformed_xtest=pd.DataFrame(data_transformed_xtest,columns=pred_var)
data_transformed_xtest.head()

,Dependents,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Self_Employed_0,Married_0,Gender_0,Gender_1,Gender_2
0,0.0,0.0,0.681284,0.0,1.426199,360.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.317267,0.0,-0.108986,360.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,2.0,0.0,0.306361,1447.0,0.329638,360.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.209732,0.0,-1.033230,360.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.195851,0.0,-1.048895,360.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [27]:
y_train=pd.DataFrame(y_train).reset_index(drop=True)
y_train.columns=['Loan_Status']

y_test=pd.DataFrame(y_test).reset_index(drop=True)
y_test.columns=['Loan_Status']

y_train=preprocess.encodingTargetVariable(y_train)
y_test=preprocess.encodingTargetVariable(y_test)

### Applying BaggingClassifier

In [28]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

In [29]:
BagCla = BaggingClassifier(base_estimator=LogisticRegression(C=10,penalty='l2'),n_estimators=10, 
                           random_state=0,max_features=0.5,max_samples=0.8)
BagCla.fit(data_transformed_xtrain, y_train)

BaggingClassifier(base_estimator=LogisticRegression(C=10, class_weight=None,
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='auto',
                                                    n_jobs=None, penalty='l2',
                                                    random_state=None,
                                                    solver='lbfgs', tol=0.0001,
                                                    verbose=0,
                                                    warm_start=False),
                  bootstrap=True, bootstrap_features=False, max_features=0.5,
                  max_samples=0.8, n_estimators=10, n_jobs=None,
                  oob_score=False, random_state=0, ver

In [30]:
BagCla.predict(data_transformed_xtrain)

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,

In [31]:
BagCla.predict_proba(data_transformed_xtrain)

array([[0.5073479 , 0.4926521 ],
       [0.2153972 , 0.7846028 ],
       [0.2543824 , 0.7456176 ],
       [0.21281919, 0.78718081],
       [0.27263377, 0.72736623],
       [0.4340327 , 0.5659673 ],
       [0.20769771, 0.79230229],
       [0.45784491, 0.54215509],
       [0.21220885, 0.78779115],
       [0.21800713, 0.78199287],
       [0.25607093, 0.74392907],
       [0.25110747, 0.74889253],
       [0.50230883, 0.49769117],
       [0.57078699, 0.42921301],
       [0.21183114, 0.78816886],
       [0.48661284, 0.51338716],
       [0.17611789, 0.82388211],
       [0.50372105, 0.49627895],
       [0.2230385 , 0.7769615 ],
       [0.22544568, 0.77455432],
       [0.20240887, 0.79759113],
       [0.50585707, 0.49414293],
       [0.25042068, 0.74957932],
       [0.26365778, 0.73634222],
       [0.26005802, 0.73994198],
       [0.48261618, 0.51738382],
       [0.2508191 , 0.7491809 ],
       [0.25835377, 0.74164623],
       [0.21103854, 0.78896146],
       [0.25124208, 0.74875792],
       [0.